In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'

# data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df = [train_df, test_df]
submit_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

# Look At Data

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
train_df.head()

In [ ]:
train_df.info()

# EDA

what is in f_27?

In [ ]:
train_df['f_27'].head(20)

its basically just a 10 character string with random characters from a-z

plan: label encode the string for the values of each character in it

look for normal distribution in numerical data

In [ ]:
col_index = 1
plt.figure(figsize=(15,10))
for col in train_df.columns:
    if ((col != 'id') & (col != 'target') & (col != 'f_27')):
        plt.subplot(5,6, col_index)
        sns.histplot(data=train_df, x=col, bins= 100)
        col_index+=1

float datatypes are normally distributed, integer ones are not

look at correlations

In [ ]:
plt.subplots(figsize=(20,10))
mask = np.triu(np.ones_like(train_df.corr().round(2)))
sns.heatmap(train_df.corr().round(2), mask=mask, cmap='cool', annot=True)

no feature is very correlated to target, some are very correlated to each other

# Preprocess

First, label encode f_27

In [ ]:
for X in df:
    for i in range(10):
        X['f_27_'+str(i)] = X.f_27.str.get(i).apply(ord) - ord('A')


In [ ]:
train_df.head()

The [tutorial](https://www.kaggle.com/code/paultimothymooney/getting-started-with-tensorflow-decision-forests) creates ternary features and unique characters as shown:

it makes the dataset perform way better somehow

In [ ]:
for X in df:
    X['uniqchars'] = X.f_27.apply(lambda s: len(set(s)))
    X['i_02_21'] = (X.f_21 + X.f_02 > 5.2).astype(int) - (X.f_21 + X.f_02 < -5.3).astype(int)
    X['i_05_22'] = (X.f_22 + X.f_05 > 5.1).astype(int) - (X.f_22 + X.f_05 < -5.4).astype(int)
    i_00_01_26 = X.f_00 + X.f_01 + X.f_26
    X['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

Scale float features

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# #since all data is numerical just use this for all columns
# sscaler = StandardScaler()
# mmscaler = MinMaxScaler(feature_range=(-1., 1.))

# num_cols = [cname for cname in test_df.columns if 
#                     test_df[cname].dtype in ['float']]

# print(num_cols)

# for X in df:
#     X[num_cols]= X[num_cols].astype('float')

# train_df[num_cols] = sscaler.fit_transform(train_df[num_cols])
# test_df[num_cols] =sscaler.transform(test_df[num_cols])
        
# train_df.head()

check correlations after preprocessing

In [ ]:
plt.subplots(figsize=(25,10))
sns.heatmap(train_df.corr().round(2), cmap='cool', annot=True)

The new features do associate strongly with target

Delete unnessesary features

In [ ]:
bad_cols = ['id', 'f_27']

train_df.drop(bad_cols, axis=1, inplace = True)
test_df.drop(bad_cols, axis=1, inplace = True)
train_df.head(1)
test_df.head(1)

# Modeling

try out many different models

In [ ]:
#import
from sklearn.model_selection import cross_val_score,train_test_split
import lightgbm as lgb
import xgboost as xgb

#Common Model Algorithms
def warn(*args, **kwargs):
    pass
import warnings
from lightgbm import LGBMClassifier

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df.drop('target',axis = 1).copy(), 
                                                      train_df.target, 
                                                      test_size=0.05, 
                                                      random_state=0)
X_test = test_df.copy()

In [ ]:
print(len(X_valid))

Try using lgbm. I have read online that it is better with dealing with large datasets and it is faster.

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [ ]:
params = {
        'task': 'prediction',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',        #auc
        'learning_rate': 0.05,  #0.05
        'max_depth': -1,        #-1
        'num_leaves': 36,       #36
        'min_data_in_leaf': 20, #20
        'colsample_bytree': 1.0,#1.0
        'lambda_l1': 0.8,       #0.8
        'lambda_l2': 0.8,       #0.8
        'num_iterations': 20000,#20000
        'verbosity': -1
}
# 0.996891

early_stop = lgb.early_stopping(stopping_rounds =100, first_metric_only=False);

eval_verbose = lgb.log_evaluation(period=500, show_stdv=False)



model = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_valid,
    callbacks = [early_stop, eval_verbose],
)


Try using xgb( doesn't work well, comment out)

In [ ]:
# xgb_train = xgb.DMatrix(X_train, y_train)
# xgb_valid = xgb.DMatrix(X_valid, y_valid)
# xgb_test = xgb.DMatrix(X_test)

In [ ]:
# params = {
#         'task': 'prediction',
#         'boosting_type': 'gbdt',
#         'objective': 'binary:logistic',
#         'metric': 'auc',        #auc
#         'learning_rate': 0.05,  #0.05
#         'max_depth': 12,        #-1
#         'num_leaves': 36,       #36
#         'min_data_in_leaf': 20, #20
#         'colsample_bytree': 1.0,#1.0
#         'lambda_l1': 0.8,       #0.8
#         'lambda_l2': 0.8,       #0.8
#         'verbosity': 0,
# }

# model = xgb.train(
#     params,
#     xgb_train,
#     evals=[(xgb_train, 'Train'), (xgb_valid, 'Valid')],
#     num_boost_round=20000,
#     verbose_eval = 100,
#     early_stopping_rounds = 100,
# )

In [ ]:
# #get accuracy
# from sklearn.metrics import mean_absolute_error

# print('accuracy: ' + str(mean_absolute_error(model.predict(xgb_valid),y_valid)))

# Predict and Submit

In [ ]:
# for i in range(len(pred)):
#     pred[i]= int(round(pred[i]))
    
# pred.astype(bool)

In [ ]:
pred = model.predict(X_test)

In [ ]:
submit_df['target'] = pred
submit_df.to_csv('submission.csv', index = False)